# <center> Feature generation: Basic
   

In [1]:
%store -r __ipy
%store -r __da
%store -r __ml

In [2]:
__ipy

Helper ipython script loaded


In [3]:
__da

Basic Data Analysis tools was loaded


In [ ]:
__ml

In [ ]:
train_df = pd.read_pickle(root + 'train_df.pkl')
test_df = pd.read_pickle(root + 'test_df.pkl')
train_df = train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [ ]:
%%time

train_df['age'] = train_df['year_built'].max() - train_df['year_built'] + 1
test_df['age'] = test_df['year_built'].max() - test_df['year_built'] + 1

le = LabelEncoder()
train_df['primary_use'] = train_df['primary_use'].astype(str)
train_df['primary_use'] = le.fit_transform(train_df['primary_use']).astype(np.int8)

test_df['primary_use'] = test_df['primary_use'].astype(str)
test_df['primary_use'] = le.fit_transform(test_df['primary_use']).astype(np.int8)

train_df['floor_count'] = train_df['floor_count'].fillna(-999).astype(np.int16)
test_df['floor_count'] = test_df['floor_count'].fillna(-999).astype(np.int16)

train_df['year_built'] = train_df['year_built'].fillna(-999).astype(np.int16)
test_df['year_built'] = test_df['year_built'].fillna(-999).astype(np.int16)

train_df['age'] = train_df['age'].fillna(-999).astype(np.int16)
test_df['age'] = test_df['age'].fillna(-999).astype(np.int16)

train_df['cloud_coverage'] = train_df['cloud_coverage'].fillna(-999).astype(np.int16)
test_df['cloud_coverage'] = test_df['cloud_coverage'].fillna(-999).astype(np.int16) 


train_df['month_datetime'] = train_df['timestamp'].dt.month.astype(np.int8)
train_df['weekofyear_datetime'] = train_df['timestamp'].dt.weekofyear.astype(np.int8)
train_df['dayofyear_datetime'] = train_df['timestamp'].dt.dayofyear.astype(np.int16)
    
train_df['hour_datetime'] = train_df['timestamp'].dt.hour.astype(np.int8)  
train_df['day_week'] = train_df['timestamp'].dt.dayofweek.astype(np.int8)
train_df['day_month_datetime'] = train_df['timestamp'].dt.day.astype(np.int8)
train_df['week_month_datetime'] = train_df['timestamp'].dt.day/7
train_df['week_month_datetime'] = train_df['week_month_datetime'].apply(lambda x: math.ceil(x)).astype(np.int8)
    
    
test_df['month_datetime'] = test_df['timestamp'].dt.month.astype(np.int8)
test_df['weekofyear_datetime'] = test_df['timestamp'].dt.weekofyear.astype(np.int8)
test_df['dayofyear_datetime'] = test_df['timestamp'].dt.dayofyear.astype(np.int16)
    
test_df['hour_datetime'] = test_df['timestamp'].dt.hour.astype(np.int8)
test_df['day_week'] = test_df['timestamp'].dt.dayofweek.astype(np.int8)
test_df['day_month_datetime'] = test_df['timestamp'].dt.day.astype(np.int8)
test_df['week_month_datetime'] = test_df['timestamp'].dt.day/7
test_df['week_month_datetime'] = test_df['week_month_datetime'].apply(lambda x: math.ceil(x)).astype(np.int8)

## Generate full key for objects

In [ ]:
def to_key(train_df, test_df):
    le_key = LabelEncoder()
    train_df[KEY_NAME] = train_df[KEY].progress_apply(\
        lambda x: str(x['building_id']) + "_" + str(x['meter']) + "_" +   str(x['site_id']), axis=1)
    
    test_df[KEY_NAME] = test_df[KEY].progress_apply(\
        lambda x: str(x['building_id']) + "_" + str(x['meter']) + "_" +   str(x['site_id']), axis=1)
    
    le_key.fit(train_df[KEY_NAME])
    train_df[KEY_NAME] = le_key.transform(train_df[KEY_NAME])
    train_df[KEY_NAME] = train_df[KEY_NAME].astype(np.int32)
    
    test_df[KEY_NAME] = le_key.transform(test_df[KEY_NAME])
    test_df[KEY_NAME] = test_df[KEY_NAME].astype(np.int32)
    
    return train_df, test_df

train_df, test_df = to_key(train_df, test_df)

In [16]:
test_df.to_pickle(root + "test_features.pkl")
test_df.to_pickle(root + "train_features.pkl")